# 패키지 불러오기

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse)
library(skimr)
library(GGally)
library(plotly)
library(viridis)
library(caret)
library(DT)
library(data.table)
library(lightgbm)
library(xgboost)
library(kableExtra)
library(magrittr)
set.seed(0)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 데이터 불러오기 
- 각각의 데이터를 불러오도록 한다. 

In [ ]:
cat("Loading data...\n")
DATA_PATH = "../input/home-credit-default-risk/"
na_strings = c("NA", "NaN", "?", "")

# code
train = fread(paste0(DATA_PATH, "application_train.csv"), 
             stringsAsFactors = FALSE, 
             data.table = FALSE, na.strings = na_strings)

head(train)

## Target
- Target 데이터의 비율 등을 확인해보자. 

In [ ]:
train %>% 
  group_by(TARGET) %>% 
  summarise(Count = n())

- 시각화를 해보기

In [ ]:
train %>% 
  group_by(TARGET) %>% 
  summarise(Count = n() / nrow(train) * 100) %>% 
  arrange(desc(Count)) %>% 
  ungroup() %>% 
  mutate(TARGET = reorder(TARGET, Count)) %>% 
  ggplot(aes(x = TARGET, y = Count)) + 
    geom_bar(stat = "identity", fill = "#5D5BDE") + 
    geom_text(aes(x = TARGET, y = 1, label = paste0(round(Count, 2), " %")), 
              hjust = 0, vjust = .5, size = 3.5, colour = "white", fontface = "bold") + 
    coord_flip() + 
    theme_minimal()

## CODE_GENDER
- Gender 확인
- XNA는 결측치

In [ ]:
train %>% 
  filter(!is.na(TARGET), CODE_GENDER != "XNA", CNT_FAM_MEMBERS <= 4) %>% # nrow() ~ 303498
  group_by(age = -round(DAYS_BIRTH/365, 0), 
           gender = ifelse(CODE_GENDER == "M", "Male", "Female"), 
           status = ifelse(NAME_FAMILY_STATUS == "Civil marriage", "Married", 
                           ifelse(NAME_FAMILY_STATUS == "Single / not married", "Single", as.character(NAME_FAMILY_STATUS)))) %>% # select(NAME_FAMILY_STATUS)
  summarise(count = n(), 
            AVG_CREDIT = mean(AMT_CREDIT), 
            AVG_TARGET = mean(TARGET)) %>% 
  mutate(AVG_TARGET = pmin(pmax(AVG_TARGET, 0.00), 0.20) * 100) %>% 
  ggplot(aes(x = age, y = count, fill = AVG_TARGET)) + 
    geom_histogram(stat = "identity", width = 1) + 
    facet_grid(gender ~ status) + 
    scale_fill_gradient("Avg. Default Rate %", low = "white", high = "blue") + 
    labs(title = "Default Rate of Applicants (N = 303,498)", 
         subtitle = "Age, Gender, And Marriage Status", 
         caption = "Created by McBert") +   
    theme_minimal()

- F 대출자가 상대적으로 많음

In [ ]:
train %>% 
  group_by(CODE_GENDER, TARGET) %>% 
  summarise(Count = n() / nrow(train) * 100) %>% 
  arrange(desc(Count)) %>% 
  ungroup() %>%
  mutate(CODE_GENDER = reorder(CODE_GENDER, Count), 
         TARGET = as.factor(TARGET)) %>%  
  ggplot(aes(x = CODE_GENDER, y = Count, fill = TARGET)) + 
    geom_bar(stat = "identity", position = position_dodge(width = 1)) + 
    geom_text(aes(x = CODE_GENDER
                  , y = Count + 2
                  , label = paste0(round(Count, 2), " %")
                  , group = TARGET)
              , position = position_dodge(width = 1)
              , size = 3.5
              , fontface = "bold") + 
    theme_minimal()

## AMT_CREDIT
- 대출 총액
- 가설 설정: 두 그룹간의 대출 총액의 유의미한 차이가 있을까? (향후 실험 필요)
- t.test() 등 기초 통계 개념 적용 필요해 보임.. 

In [ ]:
# t.test()

In [ ]:
summary(train$AMT_CREDIT)
ggplot(train, aes(x = AMT_CREDIT)) + 
  geom_histogram(bins = 30, fill = "#5D5BDE") + 
  facet_grid( ~ TARGET) + 
  theme_minimal()

## 이상치를 제거하는 코드 
- 0과 1사이 그룹 양쪽에서 대출잔액이 많은 것을 제거한 후, 모형 개발 1
- 0과 1사이 그룹 양쪽에서 대출잔액을 제거하지 않고 모형 개발 2 


In [ ]:
# 상위 2.5 제거 후 재 시각화
up_threshold = quantile(train$AMT_CREDIT, 0.975)
print(up_threshold)

# 시각화
train %>% 
  filter(AMT_CREDIT <= up_threshold) %>% 
  ggplot(aes(x = AMT_CREDIT)) + 
    geom_histogram(bins = 30, fill = "#5D5BDE") + 
    facet_grid(CODE_GENDER ~ TARGET) + 
    theme_minimal()

- 약간 정규 분포 형태로 변형됨. 

## TARGET ~ Age, Gender, Family Status
- NAME_FAMILY_STATUS: Family status of the client
- Age
- Gender

- CNT_FAM_MEMBERS: 총 가족 수인데, 1명 이상 ~ 20명 이하의 가족들도 보임. 
- 가족은 5명까지만... 나머지는 제외

In [ ]:
table(train$CNT_FAM_MEMBERS)

In [ ]:
table(train$NAME_FAMILY_STATUS)

In [ ]:
nrow(train)

- 상대적으로 Single, Male 그룹이 다른 그룹보다 Default 비율이 높았다. 
- 또한, 상대적으로 연령대가 낮을수록 Default 비율이 높았다.

In [ ]:
train %>% 
  filter(!is.na(TARGET), CODE_GENDER != "XNA", CNT_FAM_MEMBERS <= 4) %>% # nrow() ~ 303498
  group_by(age = -round(DAYS_BIRTH/365, 0), 
           gender = ifelse(CODE_GENDER == "M", "Male", "Female"), 
           status = ifelse(NAME_FAMILY_STATUS == "Civil marriage", "Married", 
                           ifelse(NAME_FAMILY_STATUS == "Single / not married", "Single", as.character(NAME_FAMILY_STATUS)))) %>% # select(NAME_FAMILY_STATUS)
  summarise(count = n(), 
            AVG_CREDIT = mean(AMT_CREDIT), 
            AVG_TARGET = mean(TARGET)) %>% 
  mutate(AVG_TARGET = pmin(pmax(AVG_TARGET, 0.00), 0.20) * 100) %>% 
  ggplot(aes(x = age, y = count, fill = AVG_TARGET)) + 
    geom_histogram(stat = "identity", width = 1) + 
    facet_grid(gender ~ status) + 
    scale_fill_gradient("Avg. Default Rate %", low = "white", high = "blue") + 
    labs(title = "Default Rate of Applicants (N = 303,498)", 
         subtitle = "Age, Gender, And Marriage Status", 
         caption = "Created by McBert") +   
    theme_minimal()

## Dept/Incomes Ratio %
- Dept의 비율이 높으면 높으수록 Default 비율도 높을 것으로 예상할 수 있다. 실제로 그런지 확인해본다. 

In [ ]:
library(scales)
train %>% 
  filter(!is.na(TARGET), CODE_GENDER != "XNA", CNT_FAM_MEMBERS <= 4) %>% 
  mutate(CODE_GENDER = ifelse(CODE_GENDER == "M", "Male", "Female"), 
         DEPT_INCOMES_RATIO = AMT_ANNUITY/AMT_INCOME_TOTAL) %>% 
  select(CODE_GENDER, DEPT_INCOMES_RATIO, TARGET) %>%
  ggplot(aes(x = DEPT_INCOMES_RATIO, y = TARGET)) + 
    geom_smooth(se = FALSE) + 
    scale_x_continuous(name = "Debt / Incomes Ratio (%)", 
                       limits = c(0, 0.5), 
                       labels = percent_format(accuracy = 0.1)) + 
    # coord_cartesian(ylim=c(0.00, 0.15)) + 
    scale_y_continuous(name = "Avg. Default Rate (%)", 
                       breaks = seq(0, 1, 0.01), 
                       labels = percent_format(accuracy = 0.1)) + 
    facet_grid(~ CODE_GENDER) + 
    labs(title = "Default Rate by Dept/Incomes Ratio (N = 303,498)", 
         subtitle = "The Comparison between Female and Male", 
         caption = "Created by McBert") +
    theme_minimal()

## 다른 데이터 수집

In [ ]:
# code
test = fread(paste0(DATA_PATH, "application_test.csv"), 
             stringsAsFactors = FALSE, 
             data.table = FALSE, na.strings = na_strings)

head(test)

In [ ]:
# code
bureau = fread(paste0(DATA_PATH, "bureau.csv"), 
               stringsAsFactors = FALSE, 
             data.table = FALSE, na.strings = na_strings)

head(bureau)
dim(bureau)

In [ ]:
# ../input/home-credit-default-risk/bureau_balance.csv
bur_balance = fread(paste0(DATA_PATH, "bureau_balance.csv"), 
               stringsAsFactors = FALSE, 
             data.table = FALSE, na.strings = na_strings)

head(bur_balance)
dim(bur_balance)

In [ ]:
table(bur_balance$SK_ID_BUREAU) %>% head(10)

In [ ]:
table(train$SK_ID_CURR) %>% head(10)

## sum_bureau_balance
- 우선 bur_balance 데이터는 매월마다 대출 잔액을 체크하고 있다. 
- 이와 같은 데이터에 대한 설명을 이해하는 것은 쉬운 건 아니다. 
  + 이럴 때는 각 데이터에 대한 부가적인 설명이 필요하다. 
  + [Discussion: Interpreting the BUREAU_BALANCE table](https://www.kaggle.com/c/home-credit-default-risk/discussion/58445)
- 우선, STATUS는 데이터 중에서도 서열측도에 해당한다고 볼 수 있다. 
  + 따라서 문자열 데이터를 숫자로 바꾸는 코드를 진행한다. 

In [ ]:
# code
stat_fn = list(mean = mean, sd = sd)

sum_bur_balance = bur_balance %>% 
  mutate_if(is.character, funs(factor(.) %>% as.integer)) %>% 
  group_by(SK_ID_BUREAU) %>% 
  mutate(SK_ID_BUREAU = as.character(SK_ID_BUREAU)) %>% 
  summarise_all(stat_fn, na.rm = TRUE)

sum_bur_balance %>% head()

- 메모리 관리

In [ ]:
rm(bur_balance)
gc()

In [ ]:
sum_bureau = bureau %>% 
  mutate(SK_ID_BUREAU = as.character(SK_ID_BUREAU)) %>% 
  left_join(sum_bur_balance, by = "SK_ID_BUREAU") %>% 
  select(-SK_ID_BUREAU) %>% 
  mutate_if(is.character, funs(factor(.) %>% as.integer)) %>% # Ordinal Encoding
  group_by(SK_ID_CURR) %>% 
  summarise_all(stat_fn)

In [ ]:
sum_bureau %>% head()

In [ ]:
rm(bureau, sum_bur_balance)
gc()

## 데이터 합치기
- 데이터를 합친다. 
- 숫자와 문자가 겹친 ID는 조인 안됨. 형변환 필요

In [ ]:
train$SK_ID_CURR = as.character(train$SK_ID_CURR)
test$SK_ID_CURR = as.character(test$SK_ID_CURR)
sum_bureau$SK_ID_CURR = as.character(sum_bureau$SK_ID_CURR)

In [ ]:
# code
tri = 1:nrow(train)
y = train$TARGET
length(tri)
length(y)

- test 데이터와 합쳐함. 
- train 데이터 TARGET 삭제 
- 365243 days는 1000년 이상, 극단 값이기 NA로 취급
- 각 데이터상의 비율 또는 Percent를 측정하는 코드 작성. 

In [ ]:
 train_test = train %>% 
  dplyr::select(-TARGET) %>% 
  bind_rows(test) %>% 
  left_join(sum_bureau, by = "SK_ID_CURR") %>% 
  select(-SK_ID_CURR) %>%
  mutate_if(is.character, funs(factor(.) %>% as.integer)) %>% 
    mutate(na = apply(., 1, function(x) sum(is.na(x))),
         DAYS_EMPLOYED = ifelse(DAYS_EMPLOYED == 365243, NA, DAYS_EMPLOYED),
         DAYS_EMPLOYED_PERC = sqrt(DAYS_EMPLOYED / DAYS_BIRTH),
         INCOME_CREDIT_PERC = AMT_INCOME_TOTAL / AMT_CREDIT,
         INCOME_PER_PERSON = log1p(AMT_INCOME_TOTAL / CNT_FAM_MEMBERS),
         ANNUITY_INCOME_PERC = sqrt(AMT_ANNUITY / (1 + AMT_INCOME_TOTAL)),
         LOAN_INCOME_RATIO = AMT_CREDIT / AMT_INCOME_TOTAL,
         ANNUITY_LENGTH = AMT_CREDIT / AMT_ANNUITY,
         CHILDREN_RATIO = CNT_CHILDREN / CNT_FAM_MEMBERS, 
         CREDIT_TO_GOODS_RATIO = AMT_CREDIT / AMT_GOODS_PRICE,
         INC_PER_CHLD = AMT_INCOME_TOTAL / (1 + CNT_CHILDREN),
         SOURCES_PROD = EXT_SOURCE_1 * EXT_SOURCE_2 * EXT_SOURCE_3,
         CAR_TO_BIRTH_RATIO = OWN_CAR_AGE / DAYS_BIRTH,
         CAR_TO_EMPLOY_RATIO = OWN_CAR_AGE / DAYS_EMPLOYED,
         PHONE_TO_BIRTH_RATIO = DAYS_LAST_PHONE_CHANGE / DAYS_BIRTH,
         PHONE_TO_EMPLOY_RATIO = DAYS_LAST_PHONE_CHANGE / DAYS_EMPLOYED) 

# code
train_test %>% glimpse()

## FLAG_DOC 
- FLAG_DOC은 고객의 문서 제공 횟수를 의미한다. 
- 이 데이터에서 `ORGANIZATION_TYPE`에 따른 `AMT_INCOME_TOTAL` 중간값을 구하도록 했다. 

In [ ]:
docs = str_subset(names(train), "FLAG_DOC")
live = str_subset(names(train), "(?!NFLAG_)(?!FLAG_DOC)(?!_FLAG_)FLAG_")

# code
inc_by_org <- train_test %>% 
  group_by(ORGANIZATION_TYPE) %>% 
  summarise(m = median(AMT_INCOME_TOTAL)) %$% 
  setNames(as.list(m), ORGANIZATION_TYPE)

inc_by_org[1:5]

### 데이터 병합 및 Matrix 변환
- 몇가지 문법적인 내용을 소개한다. 

In [ ]:
# code
library(forcats)

set.seed(1)
level_key <- c(a = "apple", b = "banana", c = "carrot")
char_vec <- sample(c("a", "b", "c"), 10, replace = TRUE)
print(char_vec)
recode(char_vec, !!!level_key)

- named vector를 적용할 때는 !!!를 추가로 입력한다. 
  + 관련 내용: https://adv-r.hadley.nz/quasiquotation.html

- moments 패키지 내, 첨도(kurtosis)는 평균을 중심으로 얼마나 뾰족하게 분포되어 있는지를 나타냄. 양의 값일 경우 분포가 뾰족하고 음의 값일 경우 분포가 평평함.

In [ ]:
final_df <- train_test %>% 
  mutate(DOC_IND_KURT = apply(train_test[, docs], 1, moments::kurtosis),
         LIVE_IND_SUM = apply(train_test[, live], 1, sum),
         # code
         NEW_INC_BY_ORG = recode(train_test$ORGANIZATION_TYPE, !!!inc_by_org),
         NEW_EXT_SOURCES_MEAN = apply(train_test[, c("EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3")], 1, mean),
         NEW_SCORES_STD = apply(train_test[, c("EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3")], 1, sd))%>%
  mutate_all(funs(ifelse(is.nan(.), NA, .))) %>% 
  mutate_all(funs(ifelse(is.infinite(.), NA, .))) %>% 
  data.matrix()

length(final_df)

# 모델링 

## XGBoost
- R API: https://xgboost.readthedocs.io/en/latest/R-package/xgboostPresentation.html

In [ ]:
cat("Modeling Data Split ...\n")
length(final_df)

dtest <- xgb.DMatrix(data = final_df[-tri, ])
# class(dtest)
# class(final_df)

tr_te <- final_df[tri, ]
train_split <- caret::createDataPartition(y, p = 0.9, list = F) %>% c()
dtrain <- xgb.DMatrix(data = tr_te[train_split, ], label = y[train_split])
dval <- xgb.DMatrix(data = tr_te[-train_split, ], label = y[-train_split])
cols <- colnames(final_df)

- 모형학습을 진행한다. 

In [ ]:
# 파라미터 정의
p <- list(objective = "binary:logistic"
         , booster = "gbtree"
         , eval_metric = "auc"
         , nthread = 6
         , eta = 0.05
         , max_depth = 3
         , min_child_weight = 30
         , nrounds = 100)

# 모형 학습
set.seed(0)
xgb_model <- xgb.train(p, dtrain, p$nrounds, list(val = dval), print_every_n = 50, early_stopping_rounds = 300)

# (option) 변수 중요도

In [ ]:
xgb.importance(cols, model = xgb_model) %>% xgb.plot.importance(top_n = 30)

## 제출

In [ ]:
read_csv(paste0(DATA_PATH, "sample_submission.csv")) %>% 
  mutate(SK_ID_CURR = as.integer(SK_ID_CURR)
         , TARGET = predict(xgb_model, dtest)) %>% write.csv("submission.csv")

## LightGBM

- API 설명: https://lightgbm.readthedocs.io/en/latest/R/index.html
- 데이터를 준비한다. 

In [ ]:
# code

In [ ]:
# code

# 결괏값 제출

- XGBoost

In [ ]:
# read_csv(paste0(DATA_PATH, "sample_submission.csv")) %>%  
#   mutate(SK_ID_CURR = as.integer(SK_ID_CURR),
#          TARGET = predict(m_xgb, dtest)) %>%
#  write_csv("submission.csv")

- LightGBM

In [ ]:
# read_csv(paste0(DATA_PATH, "sample_submission.csv")) %>% 
#     mutate(SK_ID_CURR = as.integer(SK_ID_CURR), 
#            TARGET = predict(lgb.model, data = dtest, n = lgb.model$best_score)) %>%
#    write_csv("submission.csv")